# Entity Analysis

## Imports

In [93]:
# %%capture
!pip install IPython

In [94]:
import IPython

In [ ]:
%%capture
!pip install ipywidgets

In [ ]:
%%capture
!pip install spacy-transformers

In [2]:
import os 
import pandas as pd
from tqdm import tqdm
import json

In [1]:
from pdfminer.layout    import LAParams#, LTTextBox
from pdfminer.pdfpage   import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.converter import TextConverter
from pdfminer.layout    import LTTextBoxHorizontal,LTTextLine,LTTextLineHorizontal,LTLine,LTRect,LTFigure
from pdfminer.utils     import open_filename
from io                 import StringIO

import textdistance   
import pandas as pd
import warnings
import pdfminer       
import json

import inspect
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [3]:
from IPython.display import clear_output

In [4]:
from NER.dataloader import annotations

In [5]:
import spacy

# Data Loader

# Input data:

we have data for training seven custom entities 
each one has two kinds of data:

1. Annotations
2. Documents

**Annotations:** this is text file with comma separated lines, first part is an annotations or text and the next part is the entity name
**Documents:** are parts of the pdfs that contain the annotation and need to be recognized. this is folder with many files. Each file is a piece of text that includes one or more annotations. 


context : This big body is going to see a scientific place
            Annotations : big boy, entity name: student
            Annotation  : scientific place : school 
            

*we want to create training data with the following format:* **(context,(start_indx,end_index),entity-name)**<br>
start and end indices are those  locate annotation in the context string 


# Process

## Create records from one context 
1. Read one context
2. Read the annotations file    
    Create a record.      
    The record is created by searching and locating any annotation in the context file.      
    The record is a list of tuples.     

## For all the documents:   
Iterate ovell off the contexts in the a document 
    

## Define folder path 

In [6]:
path_to_dataset = 'D:\phoenixs3'

model1_ann = 'D:\phoenixs3\Model-1\Annotations\model-1 (1).txt'  # lines 122 in this file is messed up!
model2_ann = 'D:\phoenixs3\Model-2\Annotations\model-2 (1).txt'
model3_ann = 'D:\phoenixs3\Model-3\Annotations\model-3 (2).txt'
model4_ann = 'D:\phoenixs3\Model-4\Annotations\model-4.txt'
model5_ann = 'D:\phoenixs3\Model-5\Annotations\model-5.txt'
model6_ann = 'D:\phoenixs3\Model-6\Annotations\model-6.txt'
model7_ann = 'D:\phoenixs3\Model-7\Annotations\model-7.txt'

annotations = [model1_ann,model2_ann,model3_ann,model4_ann,model5_ann,model6_ann,model7_ann]
# import all annotations as dataframe and remove the first row ! 
df1 = pd.read_csv(model1_ann, sep=",", names=['text','entity']); df1 = df1.drop(0)
df2 = pd.read_csv(model2_ann, sep=",", names=['text','entity']); df2 = df2.drop(0)
df3 = pd.read_csv(model3_ann, sep=",", names=['text','entity']); df3 = df3.drop(0)
df4 = pd.read_csv(model4_ann, sep=",", names=['text','entity']); df4 = df4.drop(0)
df5 = pd.read_csv(model5_ann, sep=",", names=['text','entity']); df5 = df5.drop(0)
df6 = pd.read_csv(model7_ann, sep=",", names=['text','entity']); df6 = df6.drop(0)
df7 = pd.read_csv(model1_ann, sep=",", names=['text','entity']); df7 = df7.drop(0)


## load annotations

In [7]:
# get_model_docs

model1_doc = 'D:\phoenixs3\Model-1\Documents' # there are many docs in here 
model2_doc = 'D:\phoenixs3\Model-2\Documents' # there are many docs in here 
model3_doc = 'D:\phoenixs3\Model-3\Documents' # there are many docs in here 
model4_doc = 'D:\phoenixs3\Model-4\Documents' # there are many docs in here 
model5_doc = 'D:\phoenixs3\Model-5\Documents' # there are many docs in here 
model6_doc = 'D:\phoenixs3\Model-6\Documents' # there are many docs in here 
model7_doc = 'D:\phoenixs3\Model-7\Documents' # there are many docs in here 

document = {'model1_doc':model1_doc,
        'model2_doc':model2_doc,
        'model3_doc':model3_doc,
        'model4_doc':model4_doc,
        'model5_doc':model5_doc,
        'model6_doc':model6_doc,
        'model7_doc':model7_doc,
       }

In [12]:
def get_model_context_file_names(document,required_model):
    """
    Reads one context file.
    
    Parameters:
    document (dict): holds paths for all model
    required_model (string): name of the model for which we want to extract data for 
  
    Returns:
    str: a string containing the text in the pdf   
    """
    doc_path=[]
    # print(required_model)
    value = document[required_model]
    # print(value)
    docs = list(os.walk(value))    
    for idx,file in enumerate(docs[0][2]):
        file_name = os.path.join(value,file)
        doc_path.append(file_name)
    return doc_path

#test the funciton 
# paths = get_model_context_file_names(document,'model1_doc')

In [13]:
def read_context_file(path):
    with open(path,'r') as file:
        context = file.read()
        return context

In [14]:
def get_records_from_context(context,df):
    """
    Take a context string and all the entity sample with ther tag and generates (test, (start,end)index, entity-name)
    e.g.: context:  this is show a big build that sells groceries at cheal price
          text : sells groceries
          entity-name: supermarket 
          output : (context, (30,45),supermarket)
    Parameters:
    context (str): holds the info which contains the data to search in 
    df (pandas frame): is the dataframe holding all text-entity pairs in two columns
    Returns:
    str: a list of tuples(contenxt,(start-index,end-index),entity-name)   
    """
    record = []
    for index, row in df.iterrows():
        if row['text'] in context:
            
            text = row['text'].strip()
            entity_name = row['entity']
            text_length = len(text)
            start_index = context.find(text)
            end_index = start_index + text_length
            record.append((context,(start_index,end_index),entity_name))
    # clean the records: 
    return record
        

# #test function  
# ##get path
# paths = get_model_context_file_names(document,'model1_doc')

# ## get context 
# a_context = read_context_file(paths[0])
# ##get records 
# records = get_records_from_context(a_context,df1)


## Spacy format

There is another way to create the dataset.    
Since for one context, we have several entities inside we can go like :    

#### Example 1
<code style='color:yellow'>
[
    ('I want apples', {'entities': [(2, 5, 'COMMAND'), (7, 12, 'FRUIT')]})
]

</code>

#### Example 2

<code style='color:yellow'>
train_data=[
    ('Who is Nishanth?', {
        'entities': [(7, 15, 'PERSON')]
    }),
     ('Who is Kamal Khumar?', {
        'entities': [(7, 19, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    })
]
</code>

In [15]:
def get_records_Spacyformat(context,df):

    entities = []
    record = []
    for index, row in df.iterrows():
        # print(row['text'])
        if row['text'] in context:
            text = row['text'].strip()
            entity_name = row['entity']
            text_length = len(text)
            start_index = context.find(text)
            end_index = start_index + text_length
            entities.append((start_index,end_index,entity_name))
    record = [(context,{'entities':entities})]
    # record = remove_overlap(record)
    return record 

# #test function  
# ##get path
# paths = get_model_context_file_names(document,'model4_doc')

# ## get context 
# a_context = read_context_file(paths[10])
# ##get records 
# records = get_records_Spacyformat(a_context,df4)
# records

In [17]:
# records

# Find similar context 

In [16]:
import re
def remove_duplicate_context(paths):
    text = []
    for idx,path in tqdm(enumerate(paths)):
        # if idx<10:

            context = read_context_file(path)
            contenxt = re.sub('[.(?)-]', '', context)
            text.append(context)
            texts = pd.Series(text)

    dropped = texts.drop_duplicates()
    return dropped

# contexts_test = remove_duplicate_context(paths)

In [21]:
contexts_test.head()

0    309g a listed building repairs notice 309g no ...
1    calderdale customer service and communications...
2    works no 39m a tree preservation order no 39n ...
3    none k an order revoking or modifying planning...
4    309g a listed building repairs notice 309g no ...
dtype: object

In [17]:
def remove_noise_from_record(records): 
    # some times, there is text which is subset of another, 'boy is hamin' and 'this boy is hamin' 
    entities = records[0][1]['entities']
    for idx,data in enumerate(entities):

        current_s = data[0]
        current_e = data[1]
        # print('idx:',idx,(data[0],data[1]))
        # print(current_s,current_e)  
        for idx1,data1 in enumerate(entities):
            if idx != idx1:
                new_s = data1[0]
                new_e = data1[1]
                # print(new_s,new_e)  
                if current_s >= new_s and  current_e <= new_e:
                    try:
                        records[0][1]['entities'].pop(idx)
                    except: 
                        print(idx)
                    
    
    return records
                
# # test function
# paths = get_model_context_file_names(document,'model1_doc')

# ## get context 
# a_context = read_context_file(paths[10])
# ##get records
# records = get_records_Spacyformat(a_context,df1.drop_duplicates())

# if len(records)>0:
#     clean_records = remove_noise_from_record(records)
# # records
# clean_records

# Load training data 

## Select training model 

In [32]:
model_context = 'model7_doc'
model = model7_ann 

## Get DF

In [33]:
DF = pd.read_csv(model, sep=",", names=['text','entity'])
DF = DF.drop(0) ; 
DF = DF.drop_duplicates() # path  model1_ann is defined above



In [34]:
DF.head(12)

,text,entity
1,172if there are any entries,17.2_QUESTION
2,172if there are entries,17.2_QUESTION
3,172 if there are any entries,17.2_QUESTION
5,a how can copies of the entries be obtained,17.2A_QUESTION
6,172a if there are any entries how can copies o...,17.2A_QUESTION
7,172 a how can copies of the entries be obtained,17.2A_QUESTION
8,1702 a how can copies of the entries be obtained,17.2A_QUESTION
11,172a if there are entries a how can copies of ...,17.2A_QUESTION
12,172 a copies of matters entered if any can be ...,17.2A_QUESTION
14,b where can the register be inspected,17.2B_QUESTION


## Get paths to context docs 

In [35]:
paths    = get_model_context_file_names(document,model_context)
print('An example path is:\n\n',paths[1])

An example path is:

 D:\phoenixs3\Model-7\Documents\17.1_QUESTION_0222c661-1edc-4929-bdec-ab056c1695ab_1


In [36]:
contexts = remove_duplicate_context(paths)
print(f"{len(paths)} context files reduced to {contexts.shape[0]} after removing duplicates")


4782it [00:07, 644.49it/s]

4782 context files reduced to 211 after removing duplicates


## Create clean dataset

In [37]:
dataset    = []
noentities = 0
print(' now creating labesl...')
# now we have records for one document and note we have to append to dataset

for context in tqdm(contexts):
        record  =  get_records_Spacyformat(context,DF)
    # print( records[0][1]['entities'])
    
        if len(record)>0: clean_record = remove_noise_from_record(record)
        if len(clean_record)>0:  dataset.append(clean_record)
        else :
            noentities+=1

 now creating labesl...


100%|███████████████████████████████████████████████████████████████████████████████| 211/211 [00:00<00:00, 359.89it/s]


In [54]:
# dataset[10][0]

In [38]:
idx = 2
text = dataset[idx][0][0]
text
ent =  dataset[idx][0][1]['entities']
for entity in ent: 
    sx = entity[0]
    ex = entity[1]
    tex = text[sx:ex]
    print(tex)

172 if there are any entries
a how can copies of the entries be obtained
b where can the register be inspected
environmental and pollution notices 18 what outstanding statutory or informal notices have been issued by the local authority under the environmental protection act 1990 or the control of pollution act 1974 this enquiry does not cover notices under part iia or part iii of the epa to which enquiries 37 or 313 apply


<h2 style='color:red'>Save dataset to csv </h2>

In [114]:
data_df = pd.DataFrame(columns = ['label','text'])
for item in dataset:
    ent = item[0][1]['entities']
    text = item[0][0]
    for entity in ent:
        # print('\n', entity[2])
    # break
        temp_df = pd.DataFrame(columns = ['label','text'],data=[[entity[2],item]])
        data_df = pd.concat([data_df,temp_df], ignore_index=True)                  
# data_df.to_csv('model_5entities.csv')

<h2 style='color:red'>Save dataset in Json</h2>

In [39]:
with open('dataset_model7.json','w') as file:
    json.dump(dataset,file)

In [40]:
with open('dataset_model6.json','r') as file:
    test_opn_Dataset = json.load(file)

In [42]:
len(test_opn_Dataset)

627

In [60]:
test_opn_Dataset[310]

[['r london borough of local land charges richmond upon thames london borough of richmond upon thames informative the historic buildings and monumentscommissior also called english heritage also have power to issue building preservation notices for listed buildings in london boroughs and enquiry should also be made of them if appropriate in respect of 39m please contact at the trees team to ascertain the status of any tree preservation order revealed in the answer trees@richmondgovuk or 020 8891 1411 310 community infrastructure levy cil a is there a cil charging schedule yes ttp//wwwrichmondgovuk/community_infrastructure_levy b if yes do any of the following subsist in relation to the property or has a local authority decided to issue serve make or commence any of the following none c has any demand notice been suspended none d has the local authority received full or part payment of any cil liability none e has the local authority received any appeal against any of the above none f h

# Go for simple matching

In [ ]:
# we have a list of all entities that can be detected in a special formats 
# lets see if we can match them 

In [ ]:
#get the entities 
we have a set of entities 
we want to search that in a set of lines ! 


ok ! this is a line ! if this is the complete entit or not! but, it should be anyway 
if it is mark it 

who are you seraching ? the df 
ok then the get a tag 
and :D

#extract text from a page

# match cases 

#report the answers 

# Go for search

## load Process entity dataframes

In [94]:
path_to_dataset = 'D:\phoenixs3'

model1_ann = 'D:\phoenixs3\Model-1\Annotations\model-1 (1).txt'  # lines 122 in this file is messed up!
model2_ann = 'D:\phoenixs3\Model-2\Annotations\model-2 (1).txt'
model3_ann = 'D:\phoenixs3\Model-3\Annotations\model-3 (2).txt'
model4_ann = 'D:\phoenixs3\Model-4\Annotations\model-4.txt'
model5_ann = 'D:\phoenixs3\Model-5\Annotations\model-5.txt'
model6_ann = 'D:\phoenixs3\Model-6\Annotations\model-6.txt'
model7_ann = 'D:\phoenixs3\Model-7\Annotations\model-7.txt'

annotations = [model1_ann,model2_ann,model3_ann,model4_ann,model5_ann,model6_ann,model7_ann]
# import all annotations as dataframe and remove the first row ! 
df1 = pd.read_csv(model1_ann, sep=",", names=['text','entity']); df1 = df1.drop(0) ; df1 = df1.drop_duplicates()
df2 = pd.read_csv(model2_ann, sep=",", names=['text','entity']); df2 = df2.drop(0) ; df2 = df2.drop_duplicates()
df3 = pd.read_csv(model3_ann, sep=",", names=['text','entity']); df3 = df3.drop(0) ; df3 = df3.drop_duplicates()
df4 = pd.read_csv(model4_ann, sep=",", names=['text','entity']); df4 = df4.drop(0) ; df4 = df4.drop_duplicates()
df5 = pd.read_csv(model5_ann, sep=",", names=['text','entity']); df5 = df5.drop(0) ; df5 = df5.drop_duplicates()
df6 = pd.read_csv(model7_ann, sep=",", names=['text','entity']); df6 = df6.drop(0) ; df6 = df6.drop_duplicates()
df7 = pd.read_csv(model1_ann, sep=",", names=['text','entity']); df7 = df7.drop(0) ; df7 = df7.drop_duplicates()



In [179]:
# Append dataframes 
ent_df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)

In [96]:
#get access to one itentity
text   = pdf_df.iloc[0]['text']
entity = pdf_df.iloc[0]['entity']

# print ('\n text:   ', text,'\n\n', 'entity :' ,entity)

## Extract PDF pages 

In [97]:
from utils.extract_pdf_elements import pdfDF
columns = ['page','item','BBOX','text','size','sx','sy','ex','ey']
pdf = 'hart3.pdf'
PDF = pdfDF(pdf,columns)
pdf_df = PDF.get_DF()

Processing page 44....


In [98]:
# query page for textlines
def get_textlines_in_page(df = df, page=1):
    df_temp = df.query(f"page=={page} and item=='LTTextLineHorizontal'")
    return df_temp

page = 2
txt_lines = get_textlines_in_page(df = df, page = 1)


In [138]:
txt_lines.iloc[3]['text']
txt_lines.shape
txt_lines.head()

,page,item,BBOX,text,size,sx,sy,ex,ey
2785,23,LTTextLineHorizontal,"(152.20000000000002, 806.133, 440.718999999999...",Law Society CON29 Enquiries of Local Authority...,"[0, 0, 595, 841]",152.2,806.133,440.719,817.133
2787,23,LTTextLineHorizontal,"(40.800000000000004, 781.2834000000001, 108.84...",Property Address:,"[0, 0, 595, 841]",40.8,781.2834,108.8472,789.0834
2789,23,LTTextLineHorizontal,"(130.8, 781.2834000000001, 401.733, 789.083400...","Beech House, Ancells Business Park Harvest Cre...","[0, 0, 595, 841]",130.8,781.2834,401.733,789.0834
2791,23,LTTextLineHorizontal,"(40.800000000000004, 763.5334000000001, 51.642...",1,"[0, 0, 595, 841]",40.8,763.5334,51.642,771.3334
2793,23,LTTextLineHorizontal,"(40.800000000000004, 735.9334000000001, 62.484...",1.01,"[0, 0, 595, 841]",40.8,735.9334,62.484,743.7334


In [154]:
import re
text  = '1.01(j) building regulations approval'

line = re.sub('[().-]', ' ', text)
print(line)

101j building regulations approval


In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
# a = 0/

# for page in range (44):
# print(page)
page = 23
txt_lines = get_textlines_in_page(df = df, page = page)

for index0, row in txt_lines.iterrows():
    # print(row['text'])
    # t = re.sub('[().-]', ' ', row['text'])
    print(t)
    for index1,ent in ent_df.iterrows():
        text = ent['text']
        print(text)
        ent_text = ent['text']
        entity    = ent['entity']
        
        if (t == ent_text) :
            print(f"found {bcolors.BOLD}{row['text']}{bcolors.ENDC} {bcolors.OKBLUE } in {bcolors.ENDC } {ent_text[:100]}{bcolors.OKBLUE } with entity type {bcolors.ENDC }{entity}.")


In [114]:
ent_df.head(10)

,text,entity
0,11 planning and building regulation decisions ...,1.1_QUESTION
1,11 planning and building decisions and pending...,1.1_QUESTION
2,101 Planning and building regulation decisions...,1.1_QUESTION
3,11 planning and building regulation decisions ...,1.1_QUESTION
4,101 planning and building regulation decisions...,1.1_QUESTION
5,11 planning and building regulation decisions ...,1.1_QUESTION
6,11 which of the following relating to the prop...,1.1_QUESTION
7,11 planning and building regulation decisions ...,1.1_QUESTION
8,101 planning and building decisions and pendin...,1.1_QUESTION
9,101 planning and building regulation decisions...,1.1_QUESTION


In [ ]:
pd.to_csv(pdf_df)

# Go for spacy 

In [281]:
!pip install spacy 

     --------------------------------------- 11.9/11.9 MB 14.5 MB/s eta 0:00:00
     ------------------------------------- 181.6/181.6 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 13.1 MB/s eta 0:00:00
     -------------------------------------- 48.9/48.9 kB 415.3 kB/s eta 0:00:00
     ---------------------------------------- 94.7/94.7 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 13.3 MB/s eta 0:00:00
     ------------------------------------- 479.4/479.4 kB 15.1 MB/s eta 0:00:00
     ---------------------------------------- 58.6/58.6 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 14.0 MB/s eta 0:00:00
  Using cached click-8.1.3-py3-none-any.whl (96 kB)


In [283]:
import spacy 
import json
import pandas as pd
import os
from tqdm import tqdm
from spacy.tokens import DocBin

In [284]:
db = DocBin() # create a DocBin object

In [285]:
!python -m spacy download en_core_web_lg

     ------------------------------------ 587.7/587.7 MB 466.5 kB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [289]:
nlp = spacy.load('en_core_web_lg')

In [300]:
len(Data)

19600

In [308]:
db = DocBin() # create a DocBin object
Data_train = Data[:1000]
Data_valid = Data[19501:]
for text, annot in tqdm(Data_train): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

for text, annot in tqdm(Data_valid): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./valid.spacy") # save the docbin object

100%|█████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 860.90it/s]


In [309]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


# Test trained model 

In [2]:
pwd

'C:\\Users\\ali.nadian\\Documents\\pythonProjects\\PDFTextract'

In [6]:
nlp_ner = spacy.load("model-best")

In [7]:
doc = nlp_ner("Antiretroviral therapy (ART) is recommended for all HIV-infected\
individuals to reduce the risk of disease progression.\nART also is recommended \
for HIV-infected individuals for the prevention of transmission of HIV.\nPatients \
starting ART should be willing and able to commit to treatment and understand the\
benefits and risks of therapy and the importance of adherence. Patients may choose\
to postpone therapy, and providers, on a case-by-case basis, may elect to defer\
therapy on the basis of clinical and/or psychosocial factors.")

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

In [24]:
import re
text = "(c) a conservation area consent \
None \
(d) a certificate of lawfulness of existing use or development \
None \
(e) a certificate of lawfulness of  proposed use or development \
None \
f a certificate of lawfulness of proposed works for listed buildings \
None \
(g) a heritage partnership agreement"
line = re.sub('[.()-]', '', text)
# print(line)
doc = nlp_ner(line)

colors = {"1.1_QUESTION": "#F67DE3", "1.1A_QUESTION": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

c a conservation area consent None d a certificate of lawfulness of existing use or development None e a certificate of lawfulness of  proposed use or development None f a certificate of lawfulness of proposed works for listed buildings None g a heritage partnership agreement

# Functions

In [16]:
%%writefile NER/dataloader.py 

# write you improts here 


class annotations():
    
    def __init__(self,file_path):
        self.path = file_path 
        
    def read_file(self):
        '''
        Read annotation file 
        each line is an annotations containing 
        (text,lable) 
        text is a words that represent the entity 
        creates: a list 
        '''
        with open(self.path,'r') as file:
            
            self.raw_file = file.readlines()
   
    def sort_annotations(self):
        """
        
        """
        self.read_file()
        self.raw_file = self.raw_file[1:]
        self.sorted_data = [(
                          annotation[1].strip().split(',')[:-1][0],
                          annotation[1].strip().split(',')[-1]
                         ) for annotation in self.raw_file] 
        return self.sorted_data
    
def sum_a(a:int,b:int):
    
    return a+b

# if __name__=='__main__':    

Overwriting NER/dataloader.py


In [13]:
from NER.dataloader import sum_a

12

In [4]:
A[0]

'"11 planning and building regulation decisions and pending applications which of the following relating to the property have been granted issued or refused or where applicable are the subject of pending applications or agreements",1.1_QUESTION\n'

# Unit test 

In [1]:
# %%writefile tests/loader_test.py

import unittest

# import functions to be tested 
from NER.dataloader import sum_a

class TestSum(unittest.TestCase):
    def test_list_int(self):
        """
        Test that it can sum a list of integers
        """
        a = 10 
        b = 10 
        result = sum_a(a,b)
        self.assertEqual(result, 20) , 'bad output'

if __name__ == '__main__':
      unittest.main(argv=['first-arg-is-ignored'], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [51]:
df = pd.DataFrame({
    'brand': ['Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    'style': ['cup', 'cup', 'cup', 'pack', 'pack'],
    'rating': [4, 4, 3.5, 15, 5]})

In [52]:
df

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0


In [53]:
df1 = df.drop_duplicates()

0


TypeError: cannot unpack non-iterable int object